In [ ]:
import numpy as np
import pandas as pandas
import matplotlib.pyplot as plt
import glob
import cv2
import pickle
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
import seaborn as sns
import os
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### To be Changed

In [ ]:
print(os.listdir("D:\\malignant vs benign\\"))

In [ ]:
SIZE = 256

### To be Changed

In [ ]:
images = []
labels = []
for directory_path in glob.glob("D:/malignant vs benign/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path,"*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)
        labels.append(label)
images = np.array(images)
labels = np.array(labels)


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(labels)
labels_encoded = le.transform(labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels_encoded, train_size=0.9)

In [ ]:
print("X-Train",x_train.shape)
print("Y-Train",y_train.shape)
print("X-Test",x_test.shape)
print("Y-Test",y_test.shape)

In [ ]:
# Normalize pixel values between 0 to 1
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
# One hot encode y values for neural network
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
# Load model without classifier layers
ResNet_model = ResNet50(weights="imagenet", include_top=False, input_shape = (SIZE,SIZE,3))
ResNet_model.summary()

In [ ]:
for layer in ResNet_model.layers:
    layer.trainable = False
ResNet_model.summary()

In [ ]:
feature_extractor = ResNet_model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
X_for_RF = features

In [ ]:
RF_model = RandomForestClassifier(n_estimators=128, random_state=42)

In [ ]:
RF_model.fit(X_for_RF, y_train)

In [ ]:
X_test_features = ResNet_model.predict(x_test)

In [ ]:
X_test_features = X_test_features.reshape(X_test_features.shape[0], -1)

In [ ]:
prediction_RF = RF_model.predict(X_test_features)
#prediction_RF = le.inverse_transform(prediction_RF)
display(prediction_RF)

In [ ]:
print("Accuracy : {}".format(metrics.accuracy_score(y_test, prediction_RF)))

In [ ]:
cm = confusion_matrix(y_test, prediction_RF)
sns.heatmap(cm, annot=True)

In [ ]:
n = np.random.randint(0, x_test.shape[0])
img = x_test[n]
#display(img)
plt.imshow(img)
input_img = np.expand_dims(img, axis=0)
input_img_features = ResNet_model.predict(input_img)
input_img_features = input_img_features.reshape(input_img_features.shape[0], -1)
prediction_RF = RF_model.predict(input_img_features)[0]
prediction_RF = le.inverse_transform([prediction_RF])
print("The prediction for the image is \t {}".format(prediction_RF[0].title()))
print("The actual label for the image is \t {}".format(le.inverse_transform([labels_encoded[n]])[0].title()))